In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

#PLOTLY
import plotly
#import plotly.plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
from plotly.graph_objs import Scatter, Figure, Layout
cf.set_config_file(offline=True)

# Set figure width to 12 and height to 9
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

In [ ]:
%%time
data = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv', nrows= 5000000)

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

In [ ]:
sns.distplot(data['fare_amount'])

plt.title('Distribution of Fare Amount')

In [ ]:
%matplotlib inline
sns.boxplot(data['fare_amount'], palette="Set2" )

plt.title('Looking for Outliers with a Boxplot')

In [ ]:
# Cab rides should not have negative numbers, along with that, taxi standarad fares begin at $2.50
data = data[data['fare_amount'] > 2.5]
    
# our latitude and longitude should not be equal to 0 becuase the dataset is based in NY
data = data[data['pickup_latitude']!= 0]
data = data[data['pickup_longitude'] != 0]
data = data[data['dropoff_latitude'] != 0]
data = data[data['dropoff_longitude'] != 0]

# latitude and longitude are bounded by 90 and -90. We shouldnt have any coordiantes out of that range
data = data[(data['pickup_latitude']<=90) & (data['pickup_latitude']>=-90)]
data = data[(data['pickup_longitude']<=90) & (data['pickup_longitude']>=-90)]
data = data[(data['dropoff_latitude']<=90) & (data['dropoff_latitude']>=-90)]
data = data[(data['dropoff_longitude']<=90) & (data['dropoff_longitude']>=-90)]
    
# I dont want to include destinations that have not moved from there pickup coordinates to there dropoff coordinates
data = data[(data['pickup_latitude'] != data['dropoff_latitude']) & (data['pickup_longitude'] != data['dropoff_longitude'])]

In [ ]:
mean = np.mean(data['fare_amount'])
sd = np.std(data['fare_amount'])    
data = data[(data['fare_amount'] > mean - 3*sd) & (data['fare_amount'] < mean + 3*sd)]

In [ ]:
data.info()

In [ ]:
sns.distplot(data['fare_amount'])

plt.title('Distribution of Fare Amount after removing outliers')

In [ ]:
sns.boxplot(data['fare_amount'], palette="Set3" )
plt.xlabel('Fare Amount')
plt.title('Distribution of Fare Amount')
plt.show()

In [ ]:
def date_columns(data):
    data['key'] = pd.to_datetime(data['key'], yearfirst=True)
    data['year'] = data['key'].dt.year
    data['month'] = data['key'].dt.month
    data['day'] = data['key'].dt.day
    data['weekday'] = data['key'].dt.weekday
    data['hour'] = data['key'].dt.hour
    #data['day_of_week'] = data['key'].dt.day_name()

In [ ]:
date_columns(data)

In [ ]:
data.info()

In [ ]:
%%time
def haversine_np(lon1,lat1,lon2,lat2):
    lon1,lat1,lon2,lat2 = map(np.radians, [lon1,lat1,lon2,lat2])
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c=2*np.arcsin(np.sqrt(a))
    km=6367*c
    return km

In [ ]:
data['haversine_dist']= haversine_np(data.pickup_longitude, data.pickup_latitude, data.dropoff_longitude, data.dropoff_latitude).astype(np.float32)

In [ ]:
ml = data[:100000]

In [ ]:
from  sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.linear_model import LinearRegression 
import xgboost as xgb

In [ ]:
y = ml['fare_amount']
X = ml.drop(['fare_amount','key', 'pickup_datetime'],axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = .3)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train)
print(lm.score(X_train,y_train))
print(lm.score(X_test,y_test))

In [ ]:
y_pred = lm.predict(X_test)
lrmse = np.sqrt(metrics.mean_squared_error(y_pred, y_test))
lrmse

In [ ]:
from sklearn.ensemble import RandomForestRegressor

randomForest = RandomForestRegressor(random_state=42)
randomForest.fit(X_train, y_train)

In [ ]:
randomForestPredict = randomForest.predict(X_test)
randomForest_mse = mean_squared_error(y_test, randomForestPredict)
randomForestMSE = np.sqrt(randomForest_mse)
randomForestMSE

In [ ]:
model = RandomForestRegressor(n_estimators=25, max_features=6, max_depth=25, min_samples_split=3,
                              min_samples_leaf=2, random_state=0)

In [ ]:
randomForestImproved = model.fit(X_train,y_train)

In [ ]:
randomForestImprovedPredict = randomForestImproved.predict(X_test)
randomForestImproved_mse = mean_squared_error(y_test, randomForestImprovedPredict)
randomForestImprovedMSE = np.sqrt(randomForestImproved_mse)
randomForestImprovedMSE

In [ ]:
feature_importance = randomForestImproved.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(20,20))
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X_train.columns[sorted_idx]) 
plt.xlabel('Relative Importance')
plt.ylabel('Features')
plt.title('Variable Importance for Random Forest')
plt.show()

In [ ]:
test = pd.read_csv('../input/new-york-city-taxi-fare-prediction/test.csv')

In [ ]:
date_columns(test)

In [ ]:
test['haversine_dist']= haversine_np(test.pickup_longitude, test.pickup_latitude, test.dropoff_longitude, test.dropoff_latitude).astype(np.float32)

In [ ]:
test.head()

In [ ]:
testing = test.drop(['key','pickup_datetime'], axis = 1)
key = test['key']
submission = pd.DataFrame({"key": test['key'], "fare_amount": predicted},columns = ['key','fare_amount'])
submission.to_csv('Submission.csv', index = False)

# Another Approach

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from scipy import stats
from scipy.stats import norm, skew
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import lightgbm as lgbm
import xgboost as xgb

In [ ]:
df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows = 4000000)
test_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
df.shape, test_df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum().sort_index()/len(df)

In [ ]:
df.dropna(subset=['dropoff_latitude', 'dropoff_longitude'], inplace = True)
df.describe()

In [ ]:
df.drop(df[df['fare_amount'] < 2.5].index, axis=0, inplace = True)
df.drop(df[df['fare_amount'] > 500].index, axis=0, inplace = True)

In [ ]:
df.drop(df[df['pickup_longitude'] == 0].index, axis=0, inplace = True)
df.drop(df[df['pickup_latitude'] == 0].index, axis=0, inplace = True)
df.drop(df[df['dropoff_longitude'] == 0].index, axis=0, inplace = True)
df.drop(df[df['dropoff_latitude'] == 0].index, axis=0, inplace = True)
df.drop(df[df['passenger_count'] == 208].index, axis=0, inplace = True)
df.drop(df[df['passenger_count'] > 5].index, axis=0, inplace = True)
df.drop(df[df['passenger_count'] == 0].index, axis=0, inplace = True)

In [ ]:
df.dropna(inplace=True)

df.drop(df.index[(df.pickup_longitude < -75) | 
           (df.pickup_longitude > -72) | 
           (df.pickup_latitude < 40) | 
           (df.pickup_latitude > 42)],inplace=True)
df.drop(df.index[(df.dropoff_longitude < -75) | 
           (df.dropoff_longitude > -72) | 
           (df.dropoff_latitude < 40) | 
           (df.dropoff_latitude > 42)],inplace=True)

In [ ]:
def get_manhattan_dist(pickup_long, pickup_lat, dropoff_long, dropoff_lat):
    return ((dropoff_long - pickup_long).abs() + (dropoff_lat - pickup_lat).abs())*100

def getEuclidean_distance(pickup_long, pickup_lat, dropoff_long, dropoff_lat):
    return np.sqrt(((pickup_long-dropoff_long)**2) + ((pickup_lat-dropoff_lat)**2))

df['distance'] = get_manhattan_dist(df['pickup_longitude'],df['pickup_latitude'],df['dropoff_longitude'],df['dropoff_latitude'])

In [ ]:
data = df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','distance','fare_amount']]

In [ ]:
df.describe()

In [ ]:
X, y = data.drop('fare_amount', axis = 1), data['fare_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [ ]:
'''
lm = RandomForestRegressor()
lm = RandomForestRegressor(n_estimators=25, max_features=6, max_depth=25, min_samples_split=3, min_samples_leaf=2, random_state=0)
lm.fit(X_train,y_train)
print(lm.score(X_train,y_train))
print(lm.score(X_test,y_test))
'''

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train)
print(lm.score(X_train,y_train))
print(lm.score(X_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

randomForest = RandomForestRegressor(random_state=42)
randomForest.fit(X_train, y_train)

In [ ]:
randomForestPredict = randomForest.predict(X_test)
randomForest_mse = mean_squared_error(y_test, randomForestPredict)
randomForestMSE = np.sqrt(randomForest_mse)
randomForestMSE

In [ ]:
test_df['distance'] = get_manhattan_dist(test_df['pickup_longitude'],test_df['pickup_latitude'],test_df['dropoff_longitude'],test_df['dropoff_latitude'])

In [ ]:
test_df

In [ ]:
test_data = test_df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','distance']]

In [ ]:
LinearPredictions = lm.predict(test_data)
LinearPredictions = np.round(LinearPredictions, decimals=2)
LinearPredictions

In [ ]:
linear_submission = pd.DataFrame({"key": test_df['key'],"fare_amount": LinearPredictions},columns = ['key','fare_amount'])
linear_submission.to_csv('submission.csv', index = False)